In [173]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

# Create dataset object for dataloader for XOR dataset

In [237]:
class Iris_dataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.x = torch.tensor(data, dtype=torch.float32)
        self.y = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [238]:
data = pd.read_csv('../data/IRIS.csv')
df_norm = data[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
target= labelencoder.fit_transform(data['species'])
target = pd.DataFrame(target)
target.rename(columns = {0:'species'}, inplace = True)
df = pd.concat([df_norm, target], axis=1)

In [261]:
class ConstructNet(nn.Module):
    def __init__(self, DNA, loss_fn, input_size, output_size, outputlayer):
        super(ConstructNet, self).__init__()
        self.DNA = DNA
        self.input_size = input_size
        self.output_size = output_size
        self.layers = []
        self.loss_fn = loss_fn
        
        # Append first layer
        self.layers.append(nn.Linear(self.input_size, self.DNA[0][1]))
        self.layers.append(nn.ReLU())
        
        for i in range(1, len(self.DNA)):
            if self.DNA[i][0] == "D":
                # The input size is the output of the last layer
                tmp_input_size = self.last_layer_output_size()
                self.layers.append(nn.Linear(tmp_input_size, self.DNA[i][1]))
                self.layers.append(nn.ReLU())
            if self.DNA[i][0] == "R":
                self.layers.append(nn.Dropout(self.DNA[i][1]))
        
        # Append the output layer        
        self.layers.append(nn.Linear(self.layers[-2].out_features, self.output_size))
        #if self.output_size > 1: 
        #    self.layers.append(nn.Softmax(dim=1))
        #else:
        #    self.layers.append(nn.Sigmoid())
        self.layers.append(outputlayer)
        self.net = nn.Sequential(*self.layers)
        
    def forward(self, x):
        return self.net(x)
    '''
    Based on the layers created, find the output size of the last dense layer
    '''
    def last_layer_output_size(self):
        for layer in self.layers[::-1]:
            if isinstance(layer, nn.Linear):
                return layer.out_features
            
    def train_net(self, device, train_loader, optimizer, epoch, log_interval, print_stats):
        self.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = self.forward(data)
            loss = self.loss_fn(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0 and print_stats:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))


    def test(self, device, test_loader):
        self.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = self.forward(data)
                #print(f"output: {output}, target: {target}")
                test_loss += self.loss_fn.forward(output, target).item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        
    def count_parameters(self):
        # https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/7
        return sum(p.numel() for p in self.net.parameters() if p.requires_grad)        
   

In [271]:
DNA_list = [
    [["D",1024],["D",512],["D",128]],
    [["D",1024],["R",0.2],["D",512],["R",0.2],["D",128]],
    [["D",64],["D", 64]]
]

# Initialise hyperparameters

In [272]:
batch_size = 16
use_mps = False and torch.backends.mps.is_available()
use_cuda = True and torch.cuda.is_available()
test_batch_size = 16
epochs = 30
lr = 0.02
gamma = 0.7
seed = 1
log_interval = 100
save_model = False

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

In [274]:
train_dataset = Iris_dataset(df.values[:,:4], df['species'])
test_dataset = Iris_dataset(df.values[:,:4], df['species'])
train_loader = torch.utils.data.DataLoader(train_dataset,shuffle=True,**train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset,shuffle=True, **test_kwargs)

In [275]:
outputlayer = nn.Softmax(dim=1)
for DNA in DNA_list:
    inv_net = ConstructNet(DNA, nn.CrossEntropyLoss(), input_size=4, output_size=3, outputlayer=outputlayer)
    print(inv_net)
    optimizer = optim.Adam(inv_net.parameters(), lr=lr)
    #scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

    for epoch in range(1, epochs + 1):
        inv_net.train_net(device, train_loader, optimizer, epoch, log_interval, print_stats=True)
        inv_net.test(device, test_loader)
        #scheduler.step()

    if save_model:
        torch.save(inv_net.state_dict(), "mnist_cnn.pt")

ConstructNet(
  (loss_fn): CrossEntropyLoss()
  (net): Sequential(
    (0): Linear(in_features=4, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=3, bias=True)
    (7): Softmax(dim=1)
  )
)
Train Epoch: 1 [0/150 (0%)]	Loss: 1.098496

Test set: Average loss: 0.0819, Accuracy: 50/150 (33%)

Train Epoch: 2 [0/150 (0%)]	Loss: 1.363945

Test set: Average loss: 0.0805, Accuracy: 50/150 (33%)

Train Epoch: 3 [0/150 (0%)]	Loss: 1.051445

Test set: Average loss: 0.0805, Accuracy: 50/150 (33%)

Train Epoch: 4 [0/150 (0%)]	Loss: 1.238945

Test set: Average loss: 0.0812, Accuracy: 50/150 (33%)

Train Epoch: 5 [0/150 (0%)]	Loss: 1.301445

Test set: Average loss: 0.0819, Accuracy: 50/150 (33%)

Train Epoch: 6 [0/150 (0%)]	Loss: 1.301445

Test set: Average loss: 0.0805, Accuracy: 50/150 (33%)

Train Epo

In [253]:
inv_net.layers[-3].weight.dtype

AttributeError: 'ReLU' object has no attribute 'weight'

In [255]:
train_dataset.y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])